In [1]:
import pandas as pd 
import numpy as np
import math 
import geopandas as gpd


train_data = pd.read_csv('data/stores_train.csv')
buss_data = pd.read_csv('data/busstops_norway.csv')

def dist_to_all_km(lat, lon, df):

    # coordinates in radians
    lat1 = lat*math.pi/180
    lon1 = lon*math.pi/180
    lat2 = df['lat']*math.pi/180 # go through whole lat column
    lon2 = df['lon']*math.pi/180 # go through whole lon column

    # store original coordinates in new dataframe
    distances = pd.DataFrame()
    distances['lat'] = df['lat'].copy()
    distances['lon'] = df['lon'].copy()

    # calculate cartesian coordinates
    R = 6371 # Earth radius in km
    df['x'] = R*np.cos(lat2)*np.cos(lon2)
    df['y'] = R*np.cos(lat2)*np.sin(lon2)
    df['z'] = R*np.sin(lat2)
    x1 = R*np.cos(lat1)*np.cos(lon1)
    y1 = R*np.cos(lat1)*np.sin(lon1)
    z1 = R*np.sin(lat1)

    # calculate distance, store as new column in the distances dataframe
    distances['dist'] = np.sqrt(np.square(df['x']-x1)+np.square(df['y']-y1)+np.square(df['z']-z1))

    return distances['dist'].squeeze()

In [ ]:

def busstops_within_range(train_data):
    train_data['busstops_within_50m'] = 0
    train_data['busstops_within_100m'] = 0
    train_data['busstops_within_400m'] = 0
    train_data['busstops_within_800m'] = 0
    train_data['busstops_within_1500m'] = 0
    iter = 0

    for index in range(len(train_data)):
        one_to_all = dist_to_all_km(train_data._get_value(index, 'lat'), train_data._get_value(index, 'lon'), buss_data)
        one_to_all = one_to_all.to_frame()
        one_to_all.rename( columns={0 :'a'}, inplace=True)
        count50 = (one_to_all < 0.05).sum()
        count100 = (one_to_all < 0.1).sum()
        count400 = (one_to_all < 0.4).sum()
        count800 = (one_to_all < 0.8).sum()
        count1500 = (one_to_all < 1.5).sum()
        iter += 1
        print(iter)
        train_data._set_value(index, 'busstops_within_50m', count50)
        train_data._set_value(index, 'busstops_within_100m', count100)
        train_data._set_value(index, 'busstops_within_400m', count400)
        train_data._set_value(index, 'busstops_within_800m', count800)
        train_data._set_value(index, 'busstops_within_1500m', count1500)


In [ ]:

import pandas as pd 
import numpy as np
import math 
import geopandas as gpd
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from shapely.geometry import Point

def ckdnearest(gdA, gdB):
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

buss = ckdnearest(df, bs)
buss = buss[['store_id', 'importance_level', 'dist']]
buss
